## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-08-54-08 +0000,bbc,Bridget Phillipson enters Labour deputy contest,https://www.bbc.com/news/articles/c3rvqv9yg4eo...
1,2025-09-09-08-41-00 +0000,wsj,French Industrial Output Retreated in July,https://www.wsj.com/economy/french-industrial-...
2,2025-09-09-08-27-00 +0000,wsj,Israel Issues Evacuation Order for All of Gaza...,https://www.wsj.com/world/middle-east/israel-i...
3,2025-09-09-08-23-08 +0000,bbc,Badenoch 'worried' UK may need IMF bailout,https://www.bbc.com/news/articles/c89dx7z98g7o...
4,2025-09-09-08-21-43 +0000,nyt,Why 7-Eleven and Other Convenience Stores in J...,https://www.nytimes.com/2025/09/09/business/7-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
378,trump,37
69,new,22
34,court,15
164,killed,13
173,epstein,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
224,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...,109
272,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...,86
292,2025-09-08-11-26-15 +0000,cbc,At least 19 killed as police open fire on Nepa...,https://www.cbc.ca/news/world/nepal-deadly-pro...,84
218,2025-09-08-17-37-04 +0000,nypost,Supreme Court lets Trump fire Democratic FTC C...,https://nypost.com/2025/09/08/us-news/supreme-...,80
195,2025-09-08-18-46-52 +0000,nyt,"Trump Administration, Again, Asks Supreme Cour...",https://www.nytimes.com/2025/09/08/us/politics...,78


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
224,109,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...
272,86,2025-09-08-13-51-11 +0000,nypost,Treasury Secretary Scott Bessent threatens hou...,https://nypost.com/2025/09/08/us-news/treasury...
292,84,2025-09-08-11-26-15 +0000,cbc,At least 19 killed as police open fire on Nepa...,https://www.cbc.ca/news/world/nepal-deadly-pro...
161,54,2025-09-08-20-43-00 +0000,wsj,Supreme Court Lifts Limits on Immigration Raid...,https://www.wsj.com/us-news/law/supreme-court-...
23,44,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...
115,42,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
90,42,2025-09-09-00-10-07 +0000,nypost,Ukrainian refugee Iryna Zarutska’s father coul...,https://nypost.com/2025/09/08/us-news/ukrainia...
175,39,2025-09-08-20-12-00 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...
255,36,2025-09-08-14-55-27 +0000,nypost,Trump demands foreign firms in US ‘hire and tr...,https://nypost.com/2025/09/08/business/trump-d...
203,33,2025-09-08-18-28-37 +0000,nypost,CBS News taps former CEO of conservative thin...,https://nypost.com/2025/09/08/media/cbs-news-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
